## **Mount Drive**

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/VIT/Tamil Argumentation

/content/gdrive/MyDrive/VIT/Tamil Argumentation


## **Install**

In [1]:
pip install transformers nltk scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## **Import Libraries**

In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from copy import deepcopy

from sklearn import metrics
from sklearn.model_selection import KFold

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,AutoModel,BertTokenizer

from nltk.tokenize import word_tokenize

from sklearn.utils import resample, shuffle

## **Import Dataset**

In [6]:
df = pd.read_excel("C:/D/VIT CHN/Internships/Tamil_Argumentation/gdrive_files/Dataset_Final/Merged/Test/merged_data_test.xlsx")

In [7]:
df.head()

,S No,Tweet,Date of Tweet,Topic,Parent Tweet,Language,Quality,Stance,Argument,Comment,Responding to Tone,Discussing Writer Characteristics,Remark,Relevancy
0,1,"Bro imagine today is Friday , big star movie i...",2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Undetermined,0,1,0,0,0,Relevant
1,2,Dei unnoda akkarai TN mela not on others and w...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Against,0,1,0,1,0,Relevant
2,3,En ninga ivara matum mention panuringa naraiya...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,For,0,1,0,0,0,Relevant
3,4,What is happening in Thoothukudi is totally no...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,High,Against,1,1,0,0,0,Relevant
4,5,Ungaluku Sterlite protest prachanaya illa Bala...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,Undetermined,0,0,1,0,0,Relevant


In [30]:
df.columns

Index(['S No', 'Tweet', 'Date of Tweet', 'Topic', 'Parent Tweet', 'Language',
       'Quality', 'Stance', 'Argument', 'Comment', 'Responding to Tone',
       'Discussing Writer Characteristics', 'Remark', 'Relevancy'],
      dtype='object')

## **Load Text and Labels**

In [8]:
text = df["Tweet and Comments"].to_numpy()
dt = df["Dataset"].to_numpy()
topic = df["Topic"].to_numpy()

Language_label = df["Language"].to_numpy()
Stance_topic = df["Stance wrt content"].to_numpy()
Stance_content = df["Stance wrt topic"].to_numpy()
Quality_label = df["Quality"].to_numpy()
Argument_label = df["Argument"].to_numpy()
Comment_label = df["Comment"].to_numpy()
Writer_label = df["Discussing Writer Characteristics"].to_numpy()
Tone_label = df["Responding to Tone"].to_numpy()
Remark_label = df["Remark"].to_numpy()
Relevancy_label = df["Relevancy"].to_numpy()

## **Label Encoding**

In [9]:
encode_dict_quality = {
    "High": np.array([1, 0, 0]),
    "Med": np.array([0, 1, 0]),
    "Med ": np.array([0, 1, 0]),
    "Low": np.array([0, 0, 1]),
}

encode_dict_language = {
    "English": np.array([1, 0, 0]),
    "Tamil": np.array([0, 1, 0]),
    "Code-Mixed": np.array([0, 0, 1]),
}

encode_dict_stance = {
    "For": np.array([1, 0, 0]),
    "Against": np.array([0, 1, 0]),
    "Undetermined": np.array([0, 0, 1]),
}

encode_dict = {
    0: np.array([1, 0]),
    1: np.array([0, 1]),
}

encode_dict_relevancy = {
    "Relevant": np.array([1, 0]),
    "Irrelevant": np.array([0, 1]),
}

In [10]:
Language_label = np.array([encode_dict_language[label] for label in Language_label])
Quality_label = np.array([encode_dict_quality[label] for label in Quality_label])
Argument_label = np.array([encode_dict[label] for label in Argument_label])
Comment_label = np.array([encode_dict[label] for label in Comment_label])
Writer_label = np.array([encode_dict[label] for label in Writer_label])
Tone_label = np.array([encode_dict[label] for label in Tone_label])
Remark_label = np.array([encode_dict[label] for label in Remark_label])
Relevancy_label = np.array([encode_dict_relevancy[label] for label in Relevancy_label])
Stance_topic = np.array([encode_dict_stance[label] for label in Stance_topic])
Stance_content = np.array([encode_dict_stance[label] for label in Stance_content])

## **Pre-Config for mBERT**

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

MAX_LEN = np.max([len(x) for x in text])
MAX_LEN = np.min([MAX_LEN, 510])

BATCH_SIZE = 32
LEARNING_RATE = 1e-1

print(MAX_LEN)
print(device)

In [16]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

## **Build Dataset for mBERT**

In [17]:
class ModelDataset(Dataset):
    def __init__(self, X, y, tokenizer, max_len):
        self.max_len = max_len
        self.text = X
        self.tokenizer = tokenizer
        self.targets = y

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        text = self.text[index]

        inputs = self.tokenizer(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True)

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device),
            'mask': torch.tensor(mask, dtype=torch.long).to(device),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).to(device),
            'targets': torch.tensor(self.targets[index], dtype=torch.float).to(device),
        }

## **Build Model**

In [18]:
class CustomModel(nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()
        self.bert = AutoModel.from_pretrained('xlm-roberta-large')

        for param in self.bert.parameters():
            param.requires_grad = False

        self.out_layer = nn.Linear(1024, 3)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.bert(
            ids, token_type_ids=token_type_ids,
            attention_mask=mask, return_dict=False
        )

        output = self.out_layer(features)

        return output

## **Test Model**

In [19]:
def train(epoch, model, train_loader, loss_fn, optimizer):

    model.train()

    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()


    print(f'Epoch: {epoch + 1}, Loss:  {loss.item()}')

In [20]:
def test_data(data_loader, model):

    model.eval()
    targets = []
    outputs = []

    with torch.no_grad():

        for batch in data_loader:

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            batch_targets = batch['targets'].to(device, dtype = torch.float)

            batch_outputs = model(ids, mask, token_type_ids)

            targets.extend(batch_targets.cpu().numpy().tolist())
            outputs.extend(batch_outputs.cpu().numpy().tolist())

    return outputs, targets

# Model initialization

In [21]:
model = CustomModel().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [23]:
test_data = ModelDataset(text, Quality_label, tokenizer, MAX_LEN)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

state_dict = torch.load("C:/D/VIT CHN/Internships/Tamil_Argumentation/Models/Merged/XLM-Roberta-Large/merged_quality_roberta.pth")
model.load_state_dict(state_dict)

outputs, targets = test_data(test_loader, model)
   
model_targets = [np.argmax(x) for x in targets]
model_labels = [np.argmax(x) for x in outputs]

accuracy = metrics.accuracy_score(model_targets, model_labels)
f1_score_w_avg = metrics.f1_score(model_targets, model_labels, average='weighted')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Weighted) = {f1_score_w_avg}")

print(metrics.classification_report(model_targets, model_labels))

Split   ---->    1


100%|██████████| 42/42 [00:46<00:00,  1.11s/it]


Epoch: 1, Loss:  0.7530481219291687
[[-1.6201982498168945, 4.835434436798096], [-1.6980477571487427, 3.8664422035217285], [-5.0037665367126465, 7.2256646156311035], [-4.343851089477539, 7.133714199066162], [-1.1160944700241089, 4.236024856567383], [-2.4641897678375244, 4.0460991859436035], [-2.8874192237854004, 5.4781012535095215], [-2.0809900760650635, 3.939329147338867], [-0.6738594770431519, 1.0857552289962769], [-1.2583067417144775, 3.50996732711792], [-2.5926730632781982, 5.190237045288086], [-1.7515299320220947, 3.770350694656372], [-2.500675678253174, 5.342319011688232], [-8.200417518615723, 11.657678604125977], [-2.46620512008667, 5.147775173187256], [-3.5646259784698486, 5.166208744049072], [-3.0693492889404297, 4.49545431137085], [-2.6215667724609375, 6.040717124938965], [-2.8473541736602783, 4.822710037231445], [-1.1579258441925049, 2.763789176940918], [-5.492900848388672, 8.608266830444336], [-9.768754005432129, 15.923739433288574], [-4.744070053100586, 8.375662803649902], 

100%|██████████| 42/42 [00:45<00:00,  1.07s/it]


Epoch: 2, Loss:  0.8959667086601257
[[0.3149997591972351, -2.067988872528076], [1.204532504081726, -1.9873836040496826], [-2.521284818649292, 1.0919334888458252], [-0.7266566753387451, -0.8786175847053528], [0.7461068034172058, -2.100731372833252], [0.6345755457878113, -1.3557484149932861], [-1.0524529218673706, -0.12293757498264313], [1.3909748792648315, -1.9327938556671143], [-3.435720920562744, 4.105939865112305], [0.17025768756866455, -0.8568643927574158], [-3.092536449432373, 3.0610175132751465], [0.45465055108070374, -1.1534912586212158], [0.8436116576194763, -1.58502197265625], [-4.369581699371338, 1.579148530960083], [0.9629339575767517, -2.5324764251708984], [0.9468554854393005, -2.302129030227661], [0.011917106807231903, -0.90155428647995], [0.8491597771644592, -2.8391315937042236], [-0.16427341103553772, -0.05247314274311066], [1.7696374654769897, -2.4769840240478516], [-2.629404067993164, 0.28030556440353394], [-3.600166082382202, -0.5869195461273193], [0.019631706178188324

...

Epoch: 28, Loss:  0.7335413098335266
[[-5.18215274810791, 5.183025360107422], [-0.07995155453681946, 0.08119738101959229], [1.1250041723251343, -1.1211273670196533], [-8.287298202514648, 8.292642593383789], [-0.32787957787513733, 0.3310641646385193], [-1.9641212224960327, 1.9661540985107422], [-1.1309422254562378, 1.1349742412567139], [-0.13556423783302307, 0.1380676031112671], [0.8518205881118774, -0.851773738861084], [-4.793989181518555, 4.801928997039795], [-1.4163384437561035, 1.4188964366912842], [-1.6792184114456177, 1.680993676185608], [1.5321389436721802, -1.5308095216751099], [-1.2467858791351318, 1.2493839263916016], [-1.344767451286316, 1.345764398574829], [-6.2563395500183105, 6.261219501495361], [-8.030940055847168, 8.03547191619873], [-4.74357795715332, 4.749739646911621], [-9.222718238830566, 9.222475051879883], [-2.8976781368255615, 2.899592876434326], [-5.710960388183594, 5.714837551116943], [0.5141139030456543, -0.5114579200744629], [-8.494621276855469, 8.503367424011

100%|██████████| 42/42 [00:44<00:00,  1.06s/it]


Epoch: 29, Loss:  0.6015554666519165
[[1.9968104362487793, -1.9960013628005981], [1.067293405532837, -1.0661720037460327], [4.298031806945801, -4.29621696472168], [-1.767356514930725, 1.7699962854385376], [3.2177557945251465, -3.2160134315490723], [3.4670965671539307, -3.4670143127441406], [2.45141863822937, -2.448276996612549], [4.023873329162598, -4.021634101867676], [4.847457408905029, -4.848349571228027], [3.768336534500122, -3.763810634613037], [2.4691319465637207, -2.4664230346679688], [-0.3579905033111572, 0.3596811890602112], [6.456368446350098, -6.454133033752441], [1.0125694274902344, -1.0107935667037964], [2.4332282543182373, -2.433170795440674], [1.0017317533493042, -0.9994311332702637], [-0.003002867102622986, 0.005790233612060547], [1.410876750946045, -1.4076508283615112], [0.34718894958496094, -0.34341323375701904], [3.488039016723633, -3.486475944519043], [-0.04095087945461273, 0.042528510093688965], [4.237906455993652, -4.236142635345459], [1.54862380027771, -1.5431758

100%|██████████| 42/42 [00:44<00:00,  1.06s/it]


Epoch: 30, Loss:  0.516649603843689
[[-2.432105541229248, 2.4281420707702637], [-1.0818016529083252, 1.0826382637023926], [0.776902437210083, -0.7810310125350952], [-7.229091167449951, 7.221956253051758], [0.635939359664917, -0.6393074989318848], [-1.752145767211914, 1.7452898025512695], [-0.5803805589675903, 0.5780872106552124], [0.41440245509147644, -0.41669559478759766], [1.4398865699768066, -1.4437692165374756], [-3.1057631969451904, 3.099111557006836], [-0.7569766044616699, 0.7564884424209595], [-1.8065779209136963, 1.8068156242370605], [1.8440515995025635, -1.8495135307312012], [-2.181832790374756, 2.1806859970092773], [-1.626455307006836, 1.6218369007110596], [-5.27533483505249, 5.267726898193359], [-6.549644947052002, 6.54132604598999], [-4.5380425453186035, 4.530651092529297], [-2.637322187423706, 2.6370604038238525], [-1.380407691001892, 1.3752307891845703], [-4.964322566986084, 4.9567437171936035], [1.8379244804382324, -1.840233325958252], [-6.537404537200928, 6.527939796447

In [24]:
accuracy = metrics.accuracy_score(model_targets, model_labels)
f1_score_w_avg = metrics.f1_score(model_targets, model_labels, average='weighted')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Weighted) = {f1_score_w_avg}")

print(metrics.classification_report(model_targets, model_labels))

Accuracy Score = 0.7731397459165155
F1 Score (Weighted) = 0.7751165508879471
              precision    recall  f1-score   support

           0       0.67      0.74      0.70       606
           1       0.84      0.79      0.82      1047

    accuracy                           0.77      1653
   macro avg       0.76      0.77      0.76      1653
weighted avg       0.78      0.77      0.78      1653

